In [34]:
import pandas as pd
import json
import numpy as np
from transformers import AlbertModel, AlbertTokenizer
from transformers import BertModel, BertTokenizer
from transformers import pipeline
import torch

In [2]:
df_130 = pd.read_csv('winemag-data-130k-v2.csv')
df_130.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
df_150 = pd.read_csv('winemag-data_first150k.csv')
df_150.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [4]:
df_130 = df_130[df_150.columns]
df = pd.concat([df_130, df_150], axis = 0, ignore_index = True)
df.shape

(280901, 11)

In [5]:
df['description'].values[:10]

array(["Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",
       "This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.",
       'Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.',
       'Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.',
       "Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280901 entries, 0 to 280900
Data columns (total 11 columns):
Unnamed: 0     280901 non-null int64
country        280833 non-null object
description    280901 non-null object
designation    197701 non-null object
points         280901 non-null int64
price          258210 non-null float64
province       280833 non-null object
region_1       234594 non-null object
region_2       111464 non-null object
variety        280900 non-null object
winery         280901 non-null object
dtypes: float64(1), int64(2), object(8)
memory usage: 23.6+ MB


In [7]:
for col in ['country', 'province', 'region_1', 'variety']:
    print(col, len(df[col].unique()))

country 51
province 491
region_1 1333
variety 757


In [8]:
bert_pipe = pipeline('feature-extraction', model = BertModel.from_pretrained('bert-base-uncased'),
                     tokenizer = BertTokenizer.from_pretrained('bert-base-uncased'))
albert_pipe = pipeline('feature-extraction', model = AlbertModel.from_pretrained('albert-base-v2'), 
                       tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2'))

I1212 12:35:17.915315 140735678059392 filelock.py:254] Lock 4431903208 acquired on /Users/alecdelany/.cache/huggingface/transformers/10be6ce6d3508f1fdce98a57a574283b47c055228c1235f8686f039287ff8174.d6110e25022b713452eb83d5bfa8ae64530995a93d8e694fe52e05aa85dd3a7d.lock


I1212 12:35:18.193044 140735678059392 filelock.py:317] Lock 4431903208 released on /Users/alecdelany/.cache/huggingface/transformers/10be6ce6d3508f1fdce98a57a574283b47c055228c1235f8686f039287ff8174.d6110e25022b713452eb83d5bfa8ae64530995a93d8e694fe52e05aa85dd3a7d.lock


In [16]:
test = df['description'].values[0]
test_bert = bert_pipe(test)
test_albert = albert_pipe(test)

print(np.array(test_bert).shape)
print(np.array(test_albert).shape)

(1, 41, 768)
(1, 40, 768)


In [31]:
bert_model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenized_text = bert_tokenizer.tokenize('[CLS]' + test + '[SEP]')
indexed_tokens = bert_tokenizer.convert_tokens_to_ids(tokenized_text)

print(tokenized_text)
print(indexed_tokens)

['[CLS]', 'aroma', '##s', 'include', 'tropical', 'fruit', ',', 'broom', ',', 'br', '##im', '##stone', 'and', 'dried', 'herb', '.', 'the', 'pal', '##ate', 'isn', "'", 't', 'overly', 'expressive', ',', 'offering', 'un', '##rip', '##ened', 'apple', ',', 'citrus', 'and', 'dried', 'sage', 'alongside', 'brisk', 'acid', '##ity', '.', '[SEP]']
[101, 23958, 2015, 2421, 5133, 5909, 1010, 23528, 1010, 7987, 5714, 9221, 1998, 9550, 12810, 1012, 1996, 14412, 3686, 3475, 1005, 1056, 15241, 22570, 1010, 5378, 4895, 29443, 6675, 6207, 1010, 20418, 1998, 9550, 10878, 4077, 28022, 5648, 3012, 1012, 102]


In [58]:
bert_model.eval()
with torch.no_grad():
    outputs = bert_model(torch.tensor(indexed_tokens).unsqueeze(0))
    
print(outputs.last_hidden_state.shape)
print(outputs.pooler_output.shape)
print(np.array(list(outputs.hidden_states)).shape)

torch.Size([1, 41, 768])
torch.Size([1, 768])


ValueError: only one element tensors can be converted to Python scalars

In [66]:
albert_model = AlbertModel.from_pretrained('albert-base-v2', output_hidden_states = True)
albert_tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

tokenized_text = bert_tokenizer.tokenize('[CLS]' + test + '[SEP]')
indexed_tokens = bert_tokenizer.convert_tokens_to_ids(tokenized_text)

print(tokenized_text)
print(indexed_tokens)

['[CLS]', 'aroma', '##s', 'include', 'tropical', 'fruit', ',', 'broom', ',', 'br', '##im', '##stone', 'and', 'dried', 'herb', '.', 'the', 'pal', '##ate', 'isn', "'", 't', 'overly', 'expressive', ',', 'offering', 'un', '##rip', '##ened', 'apple', ',', 'citrus', 'and', 'dried', 'sage', 'alongside', 'brisk', 'acid', '##ity', '.', '[SEP]']
[101, 23958, 2015, 2421, 5133, 5909, 1010, 23528, 1010, 7987, 5714, 9221, 1998, 9550, 12810, 1012, 1996, 14412, 3686, 3475, 1005, 1056, 15241, 22570, 1010, 5378, 4895, 29443, 6675, 6207, 1010, 20418, 1998, 9550, 10878, 4077, 28022, 5648, 3012, 1012, 102]


In [67]:
albert_model.eval()
with torch.no_grad():
    outputs = albert_model(torch.tensor(indexed_tokens).unsqueeze(0))
    
print(outputs.last_hidden_state.shape)
print(outputs.pooler_output.shape)
print(np.array(list(outputs.hidden_states)).shape)

torch.Size([1, 41, 768])
torch.Size([1, 768])


ValueError: only one element tensors can be converted to Python scalars